Progress:

DONE:
-get package to work from command line
-generate output from fasta files
-programatically get package to work on one sequence
-figure out indexing/confirm that package runs through dataframe in order

NOT DONE:
-send outfiles files into dataframe
-get command line argument incorporated into script
-scale beyond 5 sequences
-train model with embeddings and compare results

In [123]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.ensemble
import sklearn.feature_selection
import unittest

In [124]:
import torch
import esm

In [125]:
cd /Users/loganroberts/Learn2Therm/ValidProt/notebooks

/Users/loganroberts/Learn2Therm/ValidProt/notebooks


In [126]:
#convert to pandas df
df = pd.read_csv('learn2therm_sample_50k_exploration.csv').sample(5,random_state=42)

In [127]:
df

,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
33553,33553,0.303922,0.276786,0.216783,199,0.888393,211,0.606322,221,9827,...,1004.0,26.0,50.0,24.0,MTNCKVDNKGWINVSDGSQLLDIQHLHTGFRIKDEYYDAVDDVSFE...,MDIKLHHVNKSYETGVETINVLKDIDLTIESGSWLTITGPSGSGKT...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,348,224
9427,9427,0.303371,0.173633,0.190476,178,0.572347,196,0.765625,70,875,...,1035.0,30.0,50.0,20.0,MRNVAALSMPPRVPVEEGRTFTVDEGKRLLDAVADHRMGVLVLLAL...,MSGTRPDRVIRAFVDHLVVERGMARNTVDSYARDLRRYHAHLQARG...,site-specific integrase,site-specific tyrosine recombinase XerD,256,311
199,199,0.292453,0.185629,0.184799,213,0.637725,215,0.637982,140,11324,...,989.0,32.0,54.0,22.0,MFTLKHATAASAVVLGLVFATETFAADVTWNVSLWGKRRAFTEHVE...,MKIAREFAGALAGTLAGFAMAAVTALPVFAEELKFANFTPPFHTIN...,C4-dicarboxylate TRAP transporter substrate-bi...,TRAP transporter substrate-binding protein,337,334
12447,12447,0.406250,0.369919,0.366936,221,0.898374,230,0.920000,315,14963,...,1081.0,28.0,52.5,24.5,MVKADRVHKNFGALKVLKGISLEVGRGEVLCLVGPSGSGKSTFLRC...,MPPVLEARDIAKSFGPTPALRGLSLTIDAGEVVAITGPSGSGKSTL...,amino acid ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,250,246
39489,39489,0.412371,0.207254,0.172414,111,0.575129,102,0.376384,47,12872,...,1044.0,21.5,45.0,23.5,MIISLPAALGAAVLFGVASALQAYAARTEPPGSGLRGLLRVVLRLP...,MISLTVTAVVFGVVFLAELPDKTALAGLVLGTRYRASYVFAGVAAA...,hypothetical protein,TMEM165/GDT1 family protein,271,193


In [128]:
def get_fasta_from_dataframe(
        dataframe, output_file_a: str, output_file_b: str):
    '''
    Generates fasta file type from pandas dataframe.

    Args:
        Dataframe (pandas dataframe)
        Names of output fasta files (str)

    Returns:
        Two fasta files with protein sequences and pair_id
    '''
    # meso sequence to fasta
    with open(f'{output_file_a}', 'w') as f:
        for _, row in dataframe.iterrows():
            f.write(
                '>{}\n{}\n'.format(
                    (row['prot_pair_index']),
                    row['m_protein_seq']))

    # thermo sequence to fasta
    with open(f'{output_file_b}', 'w') as f:
        for _, row in dataframe.iterrows():
            f.write(
                '>{}\n{}\n'.format(
                    (row['prot_pair_index']),
                    (row['t_protein_seq'])))

    # return output files
    return [output_file_a, output_file_b]

In [129]:
get_fasta_from_dataframe(df, 'seq_a.fasta', 'seq_b.fasta')

['seq_a.fasta', 'seq_b.fasta']

Would be a good spot to figure out how to generate esm embeddings from a fasta file.

In [132]:
#this block of code should load the files into a dataframe 

import os

# Create an empty dataframe
embeddings = pd.DataFrame(columns=['prot_pair_index', 'tensor'])

# Set the directory path containing the PyTorch files
directory = '/Users/loganroberts/Learn2Therm/ValidProt/data/pytorch_embeddings'

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.pt'):  # Adjust the file extension as needed
        file_path = os.path.join(directory, filename)
        
        # Load the PyTorch file
        data = torch.load(file_path)
        
        # Extract the tensor(s) from the loaded data
        if isinstance(data, torch.Tensor):
            tensors = [data]
        elif isinstance(data, dict):
            tensors = list(data.values())
        else:
            # Handle other cases based on your specific file structure
            continue
        
        # Append the filename and tensor(s) to the dataframe
        embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)

#expand tensor values into individual columns
embeddings = pd.concat([embeddings.drop('tensor', axis=1),
                embeddings['tensor'].apply(lambda x: pd.Series(x))], axis=1)

<ipython-input-132-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-132-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-132-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-132-2b3af67eb1b2>:29: FutureWarning: The frame.appen

In [135]:
embeddings

,prot_pair_index,0,1,2,3,4,5,6,7,8,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,170424926,-0.089057,-0.048351,-0.072258,-0.022964,0.011155,0.049804,-0.011335,-0.145744,-0.019996,...,0.141867,-0.116855,-0.058509,-0.069415,-0.059710,0.154020,-0.028967,-0.131237,-0.002997,0.196756
1,172778855,-0.004024,-0.011217,-0.035143,0.051118,-0.118710,-0.000500,-0.000316,-0.089665,-0.068695,...,0.025760,-0.009402,-0.013473,-0.062470,-0.019069,0.039852,0.118363,-0.105894,-0.026337,0.059242
2,102140025,0.049990,-0.100387,-0.035571,-0.016107,-0.132159,0.012995,0.004166,0.067679,-0.088118,...,0.035702,-0.111666,-0.110127,0.046714,-0.061923,0.065274,0.012571,-0.138126,0.054840,0.069625
3,76961413,-0.074474,-0.027899,0.005806,0.005537,-0.070316,0.009532,0.079727,-0.107065,-0.062455,...,-0.026527,-0.105266,0.017469,-0.045476,-0.106896,0.073323,0.118733,-0.177319,-0.010493,0.087448
4,9602481,0.001106,-0.054233,0.053005,0.033410,-0.124973,0.018081,0.025389,-0.076978,0.003235,...,-0.064183,-0.088116,-0.041833,-0.008492,-0.011153,-0.081377,0.110826,-0.107516,-0.019247,0.061446


In [136]:
new_df = pd.merge(embeddings, df, on='prot_pair_index')

In [137]:
new_df

,prot_pair_index,0,1,2,3,4,5,6,7,8,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
0,170424926,-0.089057,-0.048351,-0.072258,-0.022964,0.011155,0.049804,-0.011335,-0.145744,-0.019996,...,1044.0,21.5,45.0,23.5,MIISLPAALGAAVLFGVASALQAYAARTEPPGSGLRGLLRVVLRLP...,MISLTVTAVVFGVVFLAELPDKTALAGLVLGTRYRASYVFAGVAAA...,hypothetical protein,TMEM165/GDT1 family protein,271,193
1,172778855,-0.004024,-0.011217,-0.035143,0.051118,-0.118710,-0.000500,-0.000316,-0.089665,-0.068695,...,1004.0,26.0,50.0,24.0,MTNCKVDNKGWINVSDGSQLLDIQHLHTGFRIKDEYYDAVDDVSFE...,MDIKLHHVNKSYETGVETINVLKDIDLTIESGSWLTITGPSGSGKT...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,348,224
2,102140025,0.049990,-0.100387,-0.035571,-0.016107,-0.132159,0.012995,0.004166,0.067679,-0.088118,...,1035.0,30.0,50.0,20.0,MRNVAALSMPPRVPVEEGRTFTVDEGKRLLDAVADHRMGVLVLLAL...,MSGTRPDRVIRAFVDHLVVERGMARNTVDSYARDLRRYHAHLQARG...,site-specific integrase,site-specific tyrosine recombinase XerD,256,311
3,76961413,-0.074474,-0.027899,0.005806,0.005537,-0.070316,0.009532,0.079727,-0.107065,-0.062455,...,989.0,32.0,54.0,22.0,MFTLKHATAASAVVLGLVFATETFAADVTWNVSLWGKRRAFTEHVE...,MKIAREFAGALAGTLAGFAMAAVTALPVFAEELKFANFTPPFHTIN...,C4-dicarboxylate TRAP transporter substrate-bi...,TRAP transporter substrate-binding protein,337,334
4,9602481,0.001106,-0.054233,0.053005,0.033410,-0.124973,0.018081,0.025389,-0.076978,0.003235,...,1081.0,28.0,52.5,24.5,MVKADRVHKNFGALKVLKGISLEVGRGEVLCLVGPSGSGKSTFLRC...,MPPVLEARDIAKSFGPTPALRGLSLTIDAGEVVAITGPSGSGKSTL...,amino acid ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,250,246


In [67]:
# import subprocess

# # Run the command line prompt and capture the output
# output = subprocess.check_output(['esm-extract esm2_t36_3B_UR50D seq_a.fasta data/ --include mean'], universal_newlines=True)

# # Process the captured output as needed
# processed_data = output.strip()

# # Use the processed data in your script
# print("Processed data:", processed_data)


Below code is for running on individual sequences

In [38]:
# # Load ESM-2 model
# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
# batch_converter = alphabet.get_batch_converter()
# model.eval()  # disables dropout for deterministic results

# # Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
# data = [(df['prot_pair_index'].iloc[0], df['m_protein_seq'].iloc[0])]

# batch_labels, batch_strs, batch_tokens = batch_converter(data)
# batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# # Extract per-residue representations (on CPU)
# with torch.no_grad():
#     results = model(batch_tokens, repr_layers=[33], return_contacts=True)
# token_representations = results["representations"][33]

# # Generate per-sequence representations via averaging
# # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
# sequence_representations = []
# for i, tokens_len in enumerate(batch_lens):
#     sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# # Look at the unsupervised self-attention map contact predictions
# # import matplotlib.pyplot as plt
# # for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
# #     plt.matshow(attention_contacts[: tokens_len, : tokens_len])
# #     plt.title(seq)
# #     plt.show()

In [95]:
hmmer_target = pd.read_csv('protein_match_50k')
hmmer_target['protein_match'] = hmmer_target['protein_match'].map({'Yes': 1, 'No': 0})
hmmer_target



from sklearn.utils import resample

# Assuming your data is in a pandas DataFrame called 'data'
# Separate the majority and minority classes
majority_class = hmmer_target[hmmer_target['protein_match'] == 1]
minority_class = hmmer_target[hmmer_target['protein_match'] == 0]

# Undersample the majority class to match the number of minority class samples
n_samples = len(minority_class)
undersampled_majority = resample(majority_class, n_samples=n_samples, replace=False)

# Combine the undersampled majority class with the minority class
balanced_data = pd.concat([undersampled_majority, minority_class])

balanced_data

df = pd.merge(df, balanced_data, on=['prot_pair_index'])
df.shape

(17108, 33)

In [96]:
df['protein_match'].value_counts()

1    8554
0    8554
Name: protein_match, dtype: int64

In [97]:
df['structure_match'] = df['protein_match']

In [98]:
df

,Unnamed: 0_x,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,...,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len,Unnamed: 0_y,protein_match,Jaccard_Score,structure_match
0,5,0.333333,0.267380,0.254453,148,0.791444,163,0.791262,127,8919,...,MVTTGERQRNARGEGARLRLEIVAATQALLADGETATLRSIARRAG...,MNRPTYHHGDLRAAILTEAARLVAERGAERVSLRELAREAGVSHAA...,TetR/AcrR family transcriptional regulator,TetR/AcrR family transcriptional regulator,206,187,5,1,1.00,1
1,9,0.333333,0.295082,0.291498,218,0.893443,221,0.884000,158,875,...,MQIKDSVAVVTGGASGLGLATTKRLLDAGGSVVVIDLKGEDVVAEL...,MTGTVVITGGSRGIGAATAVLAAERGWQVAVSFRERRDAAEQVVRR...,3-hydroxyacyl-CoA dehydrogenase,SDR family oxidoreductase,250,244,9,1,0.75,1
2,11,0.275281,0.182156,0.152174,185,0.687732,197,0.525333,48,14963,...,MRAIGELWPTFDDVHEIAVLRGGGLGDLMFALPAIDALAAAYPEAR...,MATTTAEIGVIGGSGFYSFLDDPHEVTVQTPYGPPSDPIAVGTVAG...,glycosyltransferase family 9 protein,S-methyl-5'-thioadenosine phosphorylase,375,269,11,0,0.00,0
3,18,0.394495,0.346774,0.369099,108,0.870968,106,0.972477,130,14963,...,MHELSIAQSVVEAVCERADGRRVRSVRMVVGALCAVVPDSMLFCFE...,MGLCEAIVDAVLQRAEGRRVRSVRVRVAGHPVVREVVDQGFAMAAA...,hydrogenase maturation nickel metallochaperone...,hydrogenase maturation nickel metallochaperone...,109,124,18,1,1.00,1
4,20,0.268908,0.232727,0.247582,239,0.869091,236,0.975207,191,1674,...,MKKAVLEVRDVTKRVKGRTLLSHMSFSIYKNEICGFVGPNGSGKTT...,MNETHFGARDLSLAYGKKRIIEALNIDIPIGYITALIGRNGSGKST...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,242,275,20,1,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17103,49995,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,53,14963,...,MDNATFRLGDDLSVRLPGHSRWIGQVEREQRWLPWLAPRLPLTVST...,MPPQPRPLRPNDPREIGGFALLGRLGEGGQGTVYLGGAPDGRRVAV...,aminoglycoside phosphotransferase family protein,serine/threonine protein kinase,271,353,4875,0,0.25,0
17104,49996,0.417989,0.389163,0.392060,190,0.935961,187,0.935000,314,7134,...,MFRTGVKAEIGRSLAVVGEAEDVERAVRVVLEQRPDVVLLDVHLPG...,MILEAEPDIVVVGEAGDGEKAVEEARALQPDVVLMDIRMPRKDGVE...,response regulator transcription factor,response regulator transcription factor,200,203,4876,1,1.00,1
17105,49997,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,279,7134,...,MIRLAELTKTYPGQQHPAVDGISMEVAEGEIVVLVGPSGCGKTTTL...,MTEQPILSARGLTVDFRLRGGRRARAVDGVDLDLAPGEVLALAGES...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,331,338,4877,0,0.25,0
17106,49998,0.334764,0.331915,0.329810,232,0.987234,230,0.966387,281,11324,...,MSESHAGALLSVRGLTAGYGGATALDGVSLTVAAGETVALLGANGA...,MSLLTTSGLTRHFSGIHAVEGVDFTLEAGEIRALIGSNGAGKTTLV...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,238,235,4878,1,0.50,1


In [99]:
df_seqs = df[['m_protein_seq', 't_protein_seq', 'prot_pair_index']]
df_seqs

,m_protein_seq,t_protein_seq,prot_pair_index
0,MVTTGERQRNARGEGARLRLEIVAATQALLADGETATLRSIARRAG...,MNRPTYHHGDLRAAILTEAARLVAERGAERVSLRELAREAGVSHAA...,20752996
1,MQIKDSVAVVTGGASGLGLATTKRLLDAGGSVVVIDLKGEDVVAEL...,MTGTVVITGGSRGIGAATAVLAAERGWQVAVSFRERRDAAEQVVRR...,85102366
2,MRAIGELWPTFDDVHEIAVLRGGGLGDLMFALPAIDALAAAYPEAR...,MATTTAEIGVIGGSGFYSFLDDPHEVTVQTPYGPPSDPIAVGTVAG...,37128170
3,MHELSIAQSVVEAVCERADGRRVRSVRMVVGALCAVVPDSMLFCFE...,MGLCEAIVDAVLQRAEGRRVRSVRVRVAGHPVVREVVDQGFAMAAA...,166997534
4,MKKAVLEVRDVTKRVKGRTLLSHMSFSIYKNEICGFVGPNGSGKTT...,MNETHFGARDLSLAYGKKRIIEALNIDIPIGYITALIGRNGSGKST...,157332201
...,...,...,...
17103,MDNATFRLGDDLSVRLPGHSRWIGQVEREQRWLPWLAPRLPLTVST...,MPPQPRPLRPNDPREIGGFALLGRLGEGGQGTVYLGGAPDGRRVAV...,78849058
17104,MFRTGVKAEIGRSLAVVGEAEDVERAVRVVLEQRPDVVLLDVHLPG...,MILEAEPDIVVVGEAGDGEKAVEEARALQPDVVLMDIRMPRKDGVE...,108797464
17105,MIRLAELTKTYPGQQHPAVDGISMEVAEGEIVVLVGPSGCGKTTTL...,MTEQPILSARGLTVDFRLRGGRRARAVDGVDLDLAPGEVLALAGES...,161110219
17106,MSESHAGALLSVRGLTAGYGGATALDGVSLTVAAGETVALLGANGA...,MSLLTTSGLTRHFSGIHAVEGVDFTLEAGEIRALIGSNGAGKTTLV...,74177185


In [100]:
#get rid of stuff that isn't quantitative

df = df.drop(columns = ['Unnamed: 0_x', 'Unnamed: 0_y','thermo_index', 'm_protein_seq', 't_protein_seq',
                        'm_protein_desc', 't_protein_desc'])

In [101]:
df.columns

Index(['local_gap_compressed_percent_id', 'scaled_local_query_percent_id',
       'scaled_local_symmetric_percent_id', 'query_align_len',
       'query_align_cov', 'subject_align_len', 'subject_align_cov',
       'bit_score', 'meso_index', 'prot_pair_index', 'meso_protein_int_index',
       'thermo_protein_int_index', 'taxa_pair_index',
       'local_gap_compressed_percent_id_16s',
       'scaled_local_query_percent_id_16s',
       'scaled_local_symmetric_percent_id_16s', 'query_align_cov_16s',
       'subject_align_cov_16s', 'bit_score_16s', 'm_ogt', 't_ogt',
       'ogt_difference', 'm_protein_len', 't_protein_len', 'protein_match',
       'Jaccard_Score', 'structure_match'],
      dtype='object')

From pairplot below, it looks like query_align_cov_16s and subject_align_cov_16s dont correlate with target. Let's remove them.

In [102]:
# sns.pairplot(df.sample(500),height = 4)

In [103]:
df = df.drop(columns = ['query_align_cov_16s', 'subject_align_cov_16s'])

In [104]:
df.describe()

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,meso_index,prot_pair_index,...,scaled_local_symmetric_percent_id_16s,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_len,t_protein_len,protein_match,Jaccard_Score,structure_match
count,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,1.710800e+04,...,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000,17108.000000
mean,0.335028,0.253711,0.246046,199.578677,0.761698,198.636369,0.731170,204.140402,8156.811667,9.144065e+07,...,0.904827,1060.437807,27.391454,51.547288,24.155834,277.208090,263.709843,0.500000,0.454811,0.500000
std,0.081731,0.099072,0.101347,59.651100,0.145737,59.862260,0.190915,156.853579,4594.459364,5.221655e+07,...,0.015290,75.675183,3.687852,3.783911,3.593249,62.927629,68.167674,0.500015,0.340119,0.500015
min,0.174274,0.081301,0.045977,21.000000,0.284553,21.000000,0.104558,35.000000,5.000000,8.858000e+03,...,0.880026,891.000000,7.000000,41.000000,20.000000,30.000000,29.000000,0.000000,0.000000,0.000000
25%,0.289855,0.193427,0.189189,166.000000,0.638779,165.000000,0.612091,108.000000,4255.000000,4.693841e+07,...,0.893394,1008.000000,27.500000,50.000000,22.000000,239.000000,227.000000,0.000000,0.200000,0.000000
50%,0.318182,0.233645,0.228994,207.000000,0.763974,207.000000,0.756434,177.000000,8160.000000,9.196549e+07,...,0.903417,1055.000000,28.000000,52.500000,24.500000,270.000000,260.000000,0.500000,0.320513,0.500000
75%,0.352941,0.282258,0.273333,231.000000,0.890869,229.250000,0.891032,244.000000,12103.000000,1.361882e+08,...,0.914642,1107.000000,30.000000,52.500000,24.500000,325.000000,314.000000,1.000000,0.750000,1.000000
max,0.985075,0.985075,0.985075,398.000000,1.000000,398.000000,1.000000,1750.000000,16345.000000,1.814866e+08,...,0.996439,1679.000000,40.000000,78.000000,56.000000,400.000000,400.000000,1.000000,1.000000,1.000000


Split data into dev and test, and then split that into train and validation.

In [105]:
#drop columns that don't exihibit signficant pearson correlation with bit_score

df = df.drop(columns = ['meso_index', 'meso_protein_int_index', 'local_gap_compressed_percent_id_16s', 
                        'scaled_local_query_percent_id_16s', 'scaled_local_symmetric_percent_id_16s',
                       'bit_score_16s', 'm_ogt', 't_ogt', 'taxa_pair_index', 'thermo_protein_int_index'
                       , 'prot_pair_index', 'ogt_difference'])

In [106]:
df = df.drop(columns = ['Jaccard_Score'])


In [107]:
df.columns

Index(['local_gap_compressed_percent_id', 'scaled_local_query_percent_id',
       'scaled_local_symmetric_percent_id', 'query_align_len',
       'query_align_cov', 'subject_align_len', 'subject_align_cov',
       'bit_score', 'm_protein_len', 't_protein_len', 'protein_match',
       'structure_match'],
      dtype='object')

In [108]:
#choosing 80/20 split instead of 85/15 because of volume of data

dev, test = sklearn.model_selection.train_test_split(df, test_size=0.15, random_state=1)

train, val = sklearn.model_selection.train_test_split(dev, test_size=0.15, random_state=1)

print(dev.shape)
print(test.shape)
print(train.shape)
print(val.shape)

(14541, 12)
(2567, 12)
(12359, 12)
(2182, 12)


In [109]:
#ID target and features, separate into separate arrays

target = [columns for columns in df[['protein_match', 'structure_match']]]
input_features = [columns for columns in df.drop(columns=['protein_match', 'structure_match'])]

In [110]:
input_features

['local_gap_compressed_percent_id',
 'scaled_local_query_percent_id',
 'scaled_local_symmetric_percent_id',
 'query_align_len',
 'query_align_cov',
 'subject_align_len',
 'subject_align_cov',
 'bit_score',
 'm_protein_len',
 't_protein_len']

In [111]:
#split X and y

dev_X = dev[input_features].values
test_X = test[input_features].values

dev_y = dev[target].values
test_y = test[target].values

print(dev_X.shape, test_X.shape, dev_y.shape, test_y.shape)

(14541, 10) (2567, 10) (14541, 2) (2567, 2)


In [112]:
#same thing for training and validation data

train_X = train[input_features].values
val_X = val[input_features].values

train_y = train[target].values
val_y = val[target].values

Scale the data

In [113]:
scaler = sklearn.preprocessing.StandardScaler()
dev_X = scaler.fit_transform(dev_X)
test_X = scaler.fit_transform(test_X)
train_X = scaler.fit_transform(train_X)
val_X = scaler.fit_transform(val_X)

Train the model

In [114]:
#Random Forest

model = sklearn.ensemble.RandomForestClassifier(oob_score=True)
model.fit(dev_X, dev_y)

RandomForestClassifier(oob_score=True)

In [115]:
# Obtain predicted class probabilities
y_pred_prob = model.predict_proba(dev_X)

# Compute uncertainty estimates using out-of-bag samples
oob_predictions = model.oob_decision_function_

uncertainty = np.mean(np.var(oob_predictions, axis=1), axis=0)

# 'y_pred_prob' contains predicted class probabilities, and 'uncertainty' represents the uncertainty estimates


In [116]:
uncertainty

array([0.09550174, 0.09550174])

In [117]:
import joblib

In [118]:
joblib.dump(model, 'trained_model.pkl')

['trained_model.pkl']

In [119]:
joblib.load('trained_model.pkl')

RandomForestClassifier(oob_score=True)

Test the model, report relevant statistics

In [120]:
score = model.score(test_X, test_y)
print('Model score is: {}'.format(score))

preds = model.predict(test_X)
len(preds)

Model score is: 0.7635372029606544


2567

In [121]:
preds

array([[1, 1],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [122]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

true_labels = test_y

predicted_labels = preds

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate precision
precision = precision_score(true_labels, predicted_labels, average=None)

# Calculate recall
recall = recall_score(true_labels, predicted_labels, average=None)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average=None)

# Print the scores
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.7635372029606544
Precision: [0.80278019 0.80278019]
Recall: [0.70858896 0.70858896]
F1 Score: [0.75274949 0.75274949]
